# install

In [ ]:
!pip install tensorflow_text
# !pip install scikit-plot
# !pip install tensorflow==2.11.0
# !pip install tensorflow_text==2.11.0
# !pip install tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 

# import

In [ ]:
import zipfile, os, requests
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# os

In [ ]:
os.mkdir('old_models')
os.mkdir("predicted")

# init

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# **Try SmallBERT**

### Load models

In [ ]:
non_external_description_url = 'https://www.dropbox.com/s/rguuhi5bz033ofb/non_external_description_smallbert_L4_H512_A8.zip?dl=1'
non_external_title_url = 'https://www.dropbox.com/s/n6hmferek8iizvf/non_external_title_smallbert_L4_H512_A8.zip?dl=1'
external_title_url = 'https://www.dropbox.com/s/92a4j6z83g90dov/external_title_smallbert_L4_H512_A8.zip?dl=1'
external_description_url = 'https://www.dropbox.com/s/7mdcleynn94c32l/external_description_smallbert_L4_H512_A8.zip?dl=1'
combined_url = 'https://www.dropbox.com/s/az9aldindjvbboy/combined_smallbert_L4_H512_A8.zip?dl=1'
promise_url = 'https://www.dropbox.com/s/7ubq7pv592ewu89/promise_smallbert_L4_H512_A8.zip?dl=1'
pure_url = 'https://www.dropbox.com/s/qr3qj6ks1k97910/pure_smallbert_L4_H512_A8.zip?dl=1'

In [ ]:
def download_file_and_save(url, path):
    response = requests.get(url)
    with open(path, "wb+") as f:
        f.write(response.content)

def unzip_file(name):
  with zipfile.ZipFile(f"./old_models/{name}.zip", 'r') as zip_ref:
    zip_ref.extractall(f"./old_models/{name}")

def load_model(url):
  name = re.search('[^/\\&\?]+\.\w{3,4}(?=([\?&]).*$|$)', url)
  name = name.group()
  name = name[:-4]
  zip_path = "./old_models/"+name+".zip"
  model_path = "./old_models/"+name+"/content/"+name
  download_file_and_save(url, zip_path)
  unzip_file(name)
  model = tf.keras.models.load_model(model_path)
  return model

#Prepare test dataset

##import

In [ ]:
# Import initial files in pandas.core.frame.DataFrame

df_test = pd.read_csv('test_dataset.tsv',sep='\t', header=0)

In [ ]:
# Print shapes of datasets
print("Test:", df_test.shape)
df_test.head()
df_test.isna().sum()

ABB: (381, 2)


Text     0
Label    0
dtype: int64

In [ ]:
df_test=df_test.drop_duplicates()

print("Test2:", df_test.shape)

ABB2: (381, 2)


## clean

In [ ]:
def clean_text(s_text):
  #input / output pandas.core.series.Series
  # Step 1 - lowercase
  s_clean = s_text.str.lower()

  # Step 2 - remove urls
  s_clean = s_clean.apply(lambda x: re.sub(r'http:/\S+','link', x))
  s_clean = s_clean.apply(lambda x: re.sub(r'https:/\S+','link', x))

  # Step 3 - remove numbers and other
  s_clean = s_clean.apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

  # Step 4 - remove multi-spaces
  s_clean = s_clean.apply(lambda x: re.sub(' +', ' ', x))

  # Step 5 - tokenization
  s_clean = s_clean.apply(lambda x: word_tokenize(x))

  # Step 6 - remove short(==1) words
  s_clean = s_clean.apply(lambda x: [i for i in x if len(i)>1])
  # Final Step - convert to string
  s_clean = s_clean.apply(lambda x: " ".join(x))

  return s_clean

# Preprocess dataset
df_test['Text_clean'] = clean_text (df_test['Text'])

## dataset

In [ ]:
#prepare test dataset

ds_test = tf.data.Dataset.from_tensor_slices((df_test['Text_clean'], df_test['Label']))

ds_test = ds_test.batch(32).prefetch(tf.data.AUTOTUNE)

# Definitions

In [ ]:
def predict (model,df,ds):
  y=model.predict(ds)
  y=np.where(y > 0.0, 1, 0)

  df_pred = df.filter(items=['Text'])
  df_pred['Label']=df['Label']
  df_pred["Predicted"]=y
  return df_pred

def save_csv(df, name):
  df.to_csv('predicted/'+name+'.csv')
  return

def save_sec(df, name):
  df_text = df[df['Predicted'] == 1].filter(items=['Text'])
  df_text.to_csv('predicted/'+name+'.csv',encoding='utf-8', index=False)
  return

def test_model(name, url, df, ds):
  model=load_model(url)
  df_pred = predict(model, df, ds)

  print("Classification results for model: "+name)
  print(classification_report(df_pred['Label'], df_pred['Predicted']))

  save_csv(df_pred,name)
  #save_sec(df_pred,name+"_sec")
  return df_pred



# Experiments

In [ ]:
## 1 non_external_title

#test_model("test_non_external_title",non_external_title_url, df_test, ds_test)

In [ ]:
## 2 non_external_description
#test_model("test_non_external_description",non_external_description_url, df_test, ds_test)

In [ ]:
## 3 external_title
#test_model("test_external_title",external_title_url, df_test, ds_test)

In [ ]:
## 4 external_description
test_model("test_external_description",external_description_url, df_test, ds_test)

12/12 [==============================] - 34s 3s/step
Classification results for model: fagor_external_description
              precision    recall  f1-score   support

           0       0.86      0.38      0.53       187
           1       0.61      0.94      0.74       194

    accuracy                           0.66       381
   macro avg       0.73      0.66      0.63       381
weighted avg       0.73      0.66      0.63       381



,Text,Label,Predicted
0,Secure configurations for all Third-Party Info...,1,1
1,Network and system vulnerability assessments s...,1,1
2,Local accounts shall be disabled if not requir...,1,1
3,Third-Party shall notify GE of any separation ...,1,1
4,Accounts shall be disabled after 90 days of in...,1,1
...,...,...,...
376,This SHOULD be possible fully or semi-automati...,0,0
377,If solution is utilized to run a critical busi...,0,0
378,Solution SHOULD be able to operate without ext...,0,0
379,Ownership of software components specifically ...,0,1


In [ ]:
## 5 combo
#test_model("test_combo",combined_url, df_test, ds_test)

In [ ]:
## 6 promise
#test_model("test_promise",promise_url, df_test, ds_test)

In [ ]:
## 7 pure
test_model("test_pure",pure_url, df_test, ds_test)

12/12 [==============================] - 34s 3s/step
Classification results for model: fagor_pure
              precision    recall  f1-score   support

           0       0.66      0.46      0.54       187
           1       0.60      0.77      0.67       194

    accuracy                           0.62       381
   macro avg       0.63      0.62      0.61       381
weighted avg       0.63      0.62      0.61       381



,Text,Label,Predicted
0,Secure configurations for all Third-Party Info...,1,1
1,Network and system vulnerability assessments s...,1,1
2,Local accounts shall be disabled if not requir...,1,1
3,Third-Party shall notify GE of any separation ...,1,1
4,Accounts shall be disabled after 90 days of in...,1,1
...,...,...,...
376,This SHOULD be possible fully or semi-automati...,0,1
377,If solution is utilized to run a critical busi...,0,0
378,Solution SHOULD be able to operate without ext...,0,0
379,Ownership of software components specifically ...,0,0


# The end